In [17]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns
import datetime
from datetime import date, timedelta

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix

import pylab as pl
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import pickle

from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz 
    #In the cmd line type: conda install python-graphviz
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv("no_show_original_data_2016_with_update_weather.csv") 
df=df.drop(['Unnamed: 0'], axis=1)

### Aranging the Data

In [6]:
# הפיכת שדות מז"א למספריים
df['prcp']=df['prcp'].astype(float).astype(int)
df['temp']=df['temp'].astype(float).astype(int)

In [7]:
#הוספת שדה של יום בשבוע של הבדיקה
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['Appointment_Week_Day'] = df['AppointmentDay'].dt.dayofweek.map(dayOfWeek)

In [8]:
#הוספת שדה של המרווח בין הבדיקות
df['Delta_Dates'] = (df['AppointmentDay']-df['ScheduledDay']).dt.days
# בדיקות שתואמו אחרי הבדיקה עצמה יופיעו כמינוס 1-כי לא הגיוני
mask = df.Delta_Dates < 0
column_name = 'Delta_Dates'
df.loc[mask, column_name] = -1

In [9]:
#Get Dummies לימי השבוע של הפגישה
week_day=pd.get_dummies(df["Appointment_Week_Day"])
week_day["AppointmentID"] = df["AppointmentID"]
df=df.merge(week_day, how='left', on='AppointmentID')
df = df.drop(['Appointment_Week_Day'], axis=1)

In [10]:
# מוסיף עמודה של כמות פגישות לכל מטופל
a=df.groupby(['PatientId']).size().reset_index(name='Appointment_counts')
df=df.merge(a, how='left', on='PatientId')
#בדיקה עצמית לקוד
#df.loc[df['PatientId'] == 1.885170e+14]

## בניית דאטה אלף
1
פיצול לגילאים:
0-15 KID
16-25 YOUNG
26-40 ADOULT
41-~ OLD

2
חיתוך את כל מי שיש לו בדלתא תאריכים-פחות מ1 =אנשים שקבעו פגישה באותו יום או אחרי המועד שהתקיימה

3
הורדת שדות מיותרים
נשארנו עם :
'AppointmentDay', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'SMS_received', 'No-show', 'ScheduledWeekDay',
       'prcp', 'temp', 'Gender_', 'No-show_', 'Delta_Dates', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'Appointment_counts', 'ADOULT', 'KID', 'OLD', 'YOUNG'
4
שמירת הדאטה

In [10]:
#פיצול לגילאים
mask = df.Age < 16
column_name = 'Age_Group'
df.loc[mask, column_name] = 'KID'
mask = df.Age > 15
column_name = 'Age_Group'
df.loc[mask, column_name] = 'YOUNG'
mask = df.Age > 25
column_name = 'Age_Group'
df.loc[mask, column_name] = 'ADOULT'
mask = df.Age > 40
column_name = 'Age_Group'
df.loc[mask, column_name] = 'OLD'

#Get Dummies לגילאים
Age_Groups=pd.get_dummies(df["Age_Group"])
Age_Groups["AppointmentID"] = df["AppointmentID"]
df=df.merge(Age_Groups, how='left', on='AppointmentID')
df = df.drop(['Age_Group'], axis=1)

In [11]:
# חיתוך כל מי שיש לו דלתא קטנה מ1
df = df.drop(df[df.Delta_Dates < 1].index)

In [12]:
# הורדת שדות מיותרים
df = df.drop(columns=['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'No-show','ScheduledWeekDay',
 'Age', 'Neighbourhood', 'Neighbourhood_codes','AppointmentDay'], axis = 1)

In [15]:
df.columns

Index(['SMS_received', 'Gender_', 'Delta_Dates', 'Scholarship', 'Hipertension',
       'Diabetes', 'prcp', 'temp', 'Alcoholism', 'Handcap', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
       'Appointment_counts', 'ADOULT', 'KID', 'OLD', 'YOUNG', 'No-show_'],
      dtype='object')

In [14]:
new_index_order = [ 'SMS_received','Gender_','Delta_Dates','Scholarship', 'Hipertension', 'Diabetes','prcp',
       'temp', 'Alcoholism', 'Handcap','Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',  'Appointment_counts','ADOULT', 'KID', 'OLD', 'YOUNG', 'No-show_']
df = df.reindex(columns=new_index_order)

In [17]:
df.head()

,SMS_received,Gender_,Delta_Dates,Scholarship,Hipertension,Diabetes,prcp,temp,Alcoholism,Handcap,...,Sunday,Thursday,Tuesday,Wednesday,Appointment_counts,ADOULT,KID,OLD,YOUNG,No-show_
1,0,1,2,0,0,0,0,22,0,0,...,0,0,0,0,1,0,1,0,0,0
2,1,1,4,0,0,0,0,22,0,0,...,0,0,0,0,2,0,1,0,0,0
3,0,1,2,0,0,0,0,22,0,0,...,0,0,0,0,1,0,1,0,0,0
7,0,0,1,0,0,0,0,22,0,0,...,0,0,0,0,1,0,1,0,0,1
12,1,0,29,0,0,0,0,22,0,0,...,0,0,0,0,1,0,1,0,0,0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49281 entries, 1 to 110522
Data columns (total 24 columns):
AppointmentDay        49281 non-null datetime64[ns]
SMS_received          49281 non-null int64
Gender_               49281 non-null int8
Delta_Dates           49281 non-null int64
Scholarship           49281 non-null int64
Hipertension          49281 non-null int64
Diabetes              49281 non-null int64
prcp                  49281 non-null int32
temp                  49281 non-null int32
Alcoholism            49281 non-null int64
Handcap               49281 non-null int64
Friday                49281 non-null uint8
Monday                49281 non-null uint8
Saturday              49281 non-null uint8
Sunday                49281 non-null uint8
Thursday              49281 non-null uint8
Tuesday               49281 non-null uint8
Wednesday             49281 non-null uint8
Appointment_counts    49281 non-null int64
ADOULT                49281 non-null uint8
KID                   

In [19]:
pickle.dump(df, open("df_a.dat", "wb"))

# בניית דאטה ב
 חיתוך את כל מי שיש לו בדלתא תאריכים-פחות מ1 =אנשים שקבעו פגישה באותו יום או אחרי המועד שהתקיימה

3 הורדת שדות מיותרים נשארנו עם : 'AppointmentDay', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show', 'ScheduledWeekDay', 'prcp', 'temp', 'Gender_', 'No-show_', 'Delta_Dates', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Appointment_counts', 'ADOULT', 'KID', 'OLD', 'YOUNG' 4 שמירת הדאטה

In [2]:
df = pd.read_csv("no_show_original_data_2016.csv") 

In [3]:
#ScheduledDay קיצוץ התאריך של והופך את התאריך לסוג דייטיים
df['ScheduledDay'] =df['ScheduledDay'].apply(str)
df['ScheduledDay'] = df['ScheduledDay'].str.slice(0, 10, 1) 
df['ScheduledDay'] =  pd.to_datetime(df['ScheduledDay'], format='%Y-%m-%d')

In [4]:
#AppointmentDay הופך את התאריך לסוג דייטיים
df['AppointmentDay'] =df['AppointmentDay'].apply(str)
df['AppointmentDay'] = df['AppointmentDay'].str.slice(0, 10, 1) 
df['AppointmentDay'] =  pd.to_datetime(df['AppointmentDay'], format='%Y-%m-%d')

In [5]:
# הכנסת קודים לשכונות
df.loc[:, 'Neighbourhood_codes'] = df["Neighbourhood"].astype("category").cat.codes+1
# הפיכת שדות קטגוריים לבינריים
df.loc[:, 'Gender_'] = df["Gender"].astype("category").cat.codes
#Woman=0, Man=1
df.loc[:, 'No-show_'] = df["No-show"].astype("category").cat.codes
#show=0, no-show=1 

In [6]:
#הוספת שדה של יום בשבוע של הבדיקה
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['Appointment_Week_Day'] = df['AppointmentDay'].dt.dayofweek.map(dayOfWeek)

In [7]:
#הוספת שדה של המרווח בין הבדיקות
df['Delta_Dates'] = (df['AppointmentDay']-df['ScheduledDay']).dt.days
# בדיקות שתואמו אחרי הבדיקה עצמה יופיעו כמינוס 1-כי לא הגיוני
mask = df.Delta_Dates < 0
column_name = 'Delta_Dates'
df.loc[mask, column_name] = -1

In [11]:
# חיתוך כל מי שיש לו דלתא קטנה מ1
df = df.drop(df[df.Delta_Dates < 1].index)

In [8]:
#Get Dummies לימי השבוע של הפגישה
week_day=pd.get_dummies(df["Appointment_Week_Day"])
week_day["AppointmentID"] = df["AppointmentID"]
df=df.merge(week_day, how='left', on='AppointmentID')
df = df.drop(['Appointment_Week_Day'], axis=1)

In [10]:
# מוסיף עמודה של כמות פגישות לכל מטופל
a=df.groupby(['PatientId']).size().reset_index(name='Appointment_counts')
df=df.merge(a, how='left', on='PatientId')
#בדיקה עצמית לקוד
#df.loc[df['PatientId'] == 1.885170e+14]

In [13]:
#Get Dummies לשכונות
NC=pd.get_dummies(df["Neighbourhood_codes"])
NC["AppointmentID"] = df["AppointmentID"]
df=df.merge(NC, how='left', on='AppointmentID')

In [18]:
df["yesterday"] = df["AppointmentDay"] - timedelta(1)

In [21]:
# הורדת שדות מיותרים
#df = df.drop(columns=['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'No-show', 'Neighbourhood', 'Neighbourhood_codes','AppointmentDay'], axis = 1)

In [22]:
df.columns

Index([               'Age',        'Scholarship',       'Hipertension',
                 'Diabetes',         'Alcoholism',            'Handcap',
             'SMS_received',            'Gender_',           'No-show_',
              'Delta_Dates',             'Friday',             'Monday',
                 'Saturday',           'Thursday',            'Tuesday',
                'Wednesday', 'Appointment_counts',                    1,
                          2,                    3,                    4,
                          5,                    6,                    7,
                          8,                    9,                   10,
                         11,                   12,                   13,
                         14,                   15,                   16,
                         17,                   18,                   19,
                         20,                   21,                   22,
                         23,                   24, 

new_index_order = ['Age','Gender_','Delta_Dates','SMS_received', 'Friday', 'Monday', 'Saturday', 'Thursday', 'Tuesday', 'Wednesday','Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism',
       'Handcap','No-show_']
df = df.reindex(columns=new_index_order)

In [24]:
df.head()

,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Gender_,No-show_,Delta_Dates,...,72,73,74,75,76,77,78,79,80,81
0,0,0,0,0,0,0,0,1,0,2,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,0,4,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,1,21,...,0,0,0,0,0,0,0,0,0,0


In [137]:
pickle.dump(df, open("df_c.dat", "wb"))